In [ ]:
import nltk
from nltk import WordNetLemmatizer
from intent_cluster import generate_training_data
from gensim.models.doc2vec import Doc2Vec
import numpy as np
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from nltk.corpus.reader import wordnet
#from keras.models import Model
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import TweetTokenizer

In [ ]:
t=Tokenizer()
model=Doc2Vec.load("Models/Doc2Vec.model")
training_df=generate_training_data()
vocab_size=len(model.wv)

In [ ]:
max_len=32
def generate_embed_matrix(tkz):
    print('In the embedding function: ')
    embeddings_index={}
    f = open('models/glove.twitter.27B.100d.txt',encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
       
        embeddings_index[word] = coefs
    f.close()
    word_index = tkz.word_index
    EMBEDDING_DIM = 100 # Because we are using the 100D gloVe embeddings

Getting the embedding matrix of shape (size of word_index, embedd dimension)<br>
Hence, each word index will be embedded in a 100D

In [ ]:
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    print(embedding_matrix)
    return embedding_matrix

In [ ]:
def encode_pad_corpus():
    le=LabelEncoder()
    coded_df=pd.DataFrame(columns=['Intents','Docs'])
    
    for i in range(len(training_df['Documents'])):
        for j in range(len(training_df['Documents'][i])):
            coded_df=coded_df.append({'Intents':training_df['Intents'][i],'Docs':training_df['Documents'][i][j]},ignore_index=True)
           
    le.fit(coded_df['Intents'])
    y_data=le.transform(coded_df['Intents'])
    t.fit_on_texts(coded_df['Docs'])
    X_data=t.texts_to_sequences(coded_df['Docs'])
    x_data_pad=pad_sequences(X_data, maxlen = max_len, padding = 'post')
    print(t.word_index['macbook'])
    #print(t.word_index['would'])
    return x_data_pad,y_data,t,le

In [ ]:
def lemmatize(phrase):
    tknzr=TweetTokenizer(strip_handles=True, reduce_len=True)
    lemmatizer=WordNetLemmatizer()
    lst=tknzr.tokenize(phrase)
    lst=nltk.pos_tag([word.lower() for word in phrase])
    print(lst)
        
    lst= [lemmatizer.lemmatize(word[0],get_pos_tag(word[1])) for word in lst if get_pos_tag(word[1]) is not None]
    print(lst)
       
    return lst

In [ ]:
def get_pos_tag(tag):
        
        if tag in ['JJ', 'JJR', 'JJS']:
            return wordnet.ADJ
        elif tag in ['RB', 'RBR', 'RBS']:
            return wordnet.ADV
        elif tag in ['NN', 'NNS', 'NNP', 'NNPS']:
            return wordnet.NOUN
        elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
            return wordnet.VERB
        return None 